In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from logistic_regression import logistic_regression

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neural_network import MLPClassifier

In [4]:
data = pd.read_csv('Dataset/train.csv', index_col=0)

In [5]:
data['time'] = pd.to_datetime(
    data['time'], format='%Y%m%d')

data['day_of_year'] = data['time'].dt.dayofyear

data['day_of_year'] = (data['day_of_year']*np.pi)/180
data['sin_time'] = np.sin(data['day_of_year'])
data['cos_time'] = np.cos(data['day_of_year'])

date = data["time"]
label = data['Label']
data.drop(columns=['time', 'day_of_year', 'Label'], inplace=True)

In [5]:
data_min = data.min()
data_max = data.max()
data = (data - data_min) / (data_max - data_min)

for column in data.columns.drop(['sin_time', 'cos_time']):
    data[column + '_pow'] = np.square(data[column])

In [6]:
train_data = data[date.dt.year <= 2005]
valid_data = data[date.dt.year > 2005]

X_train = train_data
y_train = label[date.dt.year <= 2005]
X_val = valid_data
y_val = label[date.dt.year > 2005]

In [6]:
lr = logistic_regression()

In [7]:
# gammas = [0.94, 0.96, 0.98, 0.99]
# alphas = [0.0001, 0.00001, 0.00005]
# rhos = [0.95, 0.97, 0.99]
# etas = [0.1, 0.01, 0.3, 0.6]
gammas = [0.96]
alphas = [0.00001]
rhos = [0.99]
etas = [0.3]
df = pd.DataFrame(columns = ['eta', 'rho', 'gamma', 'alpha', 'loss', 'train_acc', 'valid_acc'])
for rho in rhos:
    for gamma in gammas:
        for eta in etas:      
            for alpha in alphas:
                loss = lr.fit(X_train, y_train, 2500, eta, rho, alpha, gamma, X_val, y_val)
                train_pred = lr.predict(X_train)
                train_acc = lr.accuracy(y_train, train_pred)
                val_pred = lr.predict(X_val)
                val_acc = lr.accuracy(y_val, val_pred)
                new_row = {'eta': eta, 'rho': rho, 'gamma': gamma, 'alpha': alpha, 'loss': loss, 'train_acc':train_acc, 'valid_acc': val_acc}
                df.loc[len(df)] = new_row
                df = df.sort_values(by=['loss'])
                clear_output(wait=True)
                display(df)

,eta,rho,gamma,alpha,loss,train_acc,valid_acc
0,0.3,0.99,0.96,0.00001,0.382136,0.838061,0.807923


In [1]:
rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=30, min_samples_split=10)

NameError: name 'RandomForestClassifier' is not defined

In [22]:
rfc.fit(X_train, y_train)
val_pred = rfc.predict(X_val)
print(f'Validation accuracy: {accuracy_score(y_val, val_pred)}')
train_pred = rfc.predict(X_train)
print(f'Train accuracy: {accuracy_score(y_train, train_pred)}')


Validation accuracy: 0.7937158469945356
Train accuracy: 0.8113247863247863


In [ ]:
mlp = MLPClassifier